In [ ]:
pip install mglearn

In [1]:
%matplotlib inline
from IPython.display import display
import matplotlib_inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mglearn
from cycler import cycler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import *
import os

# Libreria per importare CSV da Drive
# from google.colab import drive

matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['image.cmap'] = "viridis"
plt.rcParams['image.interpolation'] = "none"
plt.rcParams['savefig.bbox'] = "tight"
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['legend.numpoints'] = 1
plt.rc('axes', prop_cycle=(
    cycler('color', mglearn.plot_helpers.cm_cycle.colors) +
    cycler('linestyle', ['-', '-', "--", (0, (3, 3)), (0, (1.5, 1.5))])))

np.set_printoptions(precision=3, suppress=True)

pd.set_option("display.max_columns", None)
pd.set_option('display.precision', 2)

__all__ = ['np', 'mglearn', 'display', 'plt', 'pd']


In [7]:
modelli = ['LogisticRegression', 'KNeighborsClassifier',' DecisionTreeClassifier', 'RandomForestClassifier' ,'SVC', 'GaussianNB', 'LinearDiscriminantAnalysis', 'AdaBoostClassifier' ]
modelli = ['RandomForestClassifier']
def mod(modello):
    if modello == 'LogisticRegression':
        from sklearn.linear_model import LogisticRegression
        model = LogisticRegression()
    
    if modello == 'KNeighborsClassifier':
        from sklearn.neighbors import KNeighborsClassifier
        model = KNeighborsClassifier(n_neighbors=3)

    if modello == 'DecisionTreeClassifier':
        from sklearn.tree import DecisionTreeClassifier
        model = DecisionTreeClassifier()
    if modello == 'RandomForestClassifier':
        from sklearn.ensemble import RandomForestClassifier
        model = RandomForestClassifier()
    if modello == 'SVC':
        from sklearn.svm import SVC
        model = SVC(kernel='linear')
    if modello == 'GaussianNB':
        from sklearn.naive_bayes import GaussianNB
        model = GaussianNB()
    if modello == 'LinearDiscriminantAnalysis':
        from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
        model = LinearDiscriminantAnalysis()
    if modello == 'AdaBoostClassifier':
        from sklearn.ensemble import AdaBoostClassifier
        model = AdaBoostClassifier()
    return model

In [10]:
from sklearn.preprocessing import OneHotEncoder

def ohe(df, list_of_columns):
    # Inizializzare OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    
    # Manteniamo una copia del DataFrame originale
    df_encoded = df.copy()

    # Iterare sulle colonne da codificare
    for col in list_of_columns:
        # Applicare OneHotEncoder alla colonna specifica
        encoded = encoder.fit_transform(df[[col]])

        # Creare un DataFrame con le nuove colonne codificate, usando gli stessi indici del DataFrame originale
        encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([col]), index=df.index)

        # Concatenare le nuove colonne codificate al DataFrame originale
        df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
    df_encoded = df_encoded.drop(list_of_columns, axis=1)

    # Restituire il DataFrame finale, mantenendo le colonne originali
    return df_encoded

In [3]:
dfs = []
for filename in os.listdir():
    if filename.endswith('.csv'):
        print(filename)
        df_ = pd.read_csv(filename).drop(['device_fraud_count', 'month'], axis=1)
        if 'x1' in list(df_.columns):
            df_ = df_.drop(['x1', 'x2'], axis=1)
        df_['fonte'] = filename
        dfs.append(df_)
df = pd.concat(dfs)


lista_fonti = ['Base.csv', 'Variant I.csv', 'Variant II.csv','Variant III.csv','Variant IV.csv','Variant V.csv']
diz_df = dict()
for l in lista_fonti:
    diz_df[l] = df[(df['fonte'] == l)].drop(['fonte'], axis=1).drop_duplicates()

diz_df.keys()

df = diz_df['Base.csv']
df_frodi = df[(df['fraud_bool'] == 1)]
n = df[(df['fraud_bool'] == 1)].shape[0]
print(n)
df_not_frodi = df[(df['fraud_bool'] != 1)]#.head(n*50)

df = pd.concat([df_frodi, df_not_frodi])


Base.csv
11029


### Dataframe

In [4]:
display(df)

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w
43,1,0.9,5.71e-01,-1,157,60,5.91e-03,3.53e+01,AA,4079,8139.27,4312.23,6341.43,2,17,CA,131,1,BA,1,1,30,0,1500.0,0,INTERNET,4.04,windows,1,1
140,1,0.1,9.93e-01,-1,67,20,1.53e-02,9.21e+01,AA,1192,8330.73,6708.35,6709.44,37,6,CA,79,1,BB,0,1,25,0,1500.0,0,INTERNET,12.93,windows,0,1
284,1,0.8,7.94e-01,-1,376,60,1.79e+01,4.14e+01,AA,815,9747.41,6902.02,6467.15,46,3,CC,196,1,BA,1,0,28,0,1500.0,0,INTERNET,2.55,linux,0,1
314,1,0.7,9.00e-02,-1,58,50,4.13e-03,-1.24e+00,AD,4011,8036.90,5794.62,6333.34,2,6,CA,177,1,BA,0,1,27,0,1500.0,0,INTERNET,3.50,windows,0,1
561,1,0.9,9.98e-01,31,45,30,1.02e-02,3.31e+01,AA,525,6883.81,3655.04,4327.22,16,12,CA,273,1,BA,0,1,1,1,2000.0,0,INTERNET,44.54,windows,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0.8,1.25e-01,-1,143,30,5.13e-02,-8.26e-01,AB,530,6732.60,3010.05,3095.75,42,8,CA,305,1,BB,1,1,31,0,1500.0,0,INTERNET,16.97,other,0,1
999996,0,0.9,8.25e-01,-1,193,30,9.59e-03,8.31e-03,AC,408,1574.29,2716.50,4286.09,0,5,CA,235,0,BA,1,1,-1,1,1000.0,0,INTERNET,1.50,macintosh,0,1
999997,0,0.8,1.41e-01,-1,202,10,5.93e-02,5.06e+01,AA,749,1258.86,3601.32,3103.89,2,3,CA,195,1,BE,0,1,31,0,200.0,0,INTERNET,16.07,other,0,1
999998,0,0.9,2.48e-03,52,3,30,2.34e-02,-1.31e+00,AB,707,7048.14,6521.40,3068.27,7,8,CA,148,0,BD,0,1,1,0,200.0,0,INTERNET,1.38,linux,1,1


In [9]:
def train(df, model):
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuratezza: {accuracy}')

    # Calcolare la precisione
    precision = precision_score(y_test, y_pred)
    print(f'Precisione: {precision}')

    # Calcolare il recall (sensibilità)
    recall = recall_score(y_test, y_pred)
    print(f'Recall: {recall}')

    # Calcolare FPR (False Positive Ration)
    print(f'FPR: {1 - recall}')

    # Matrice di confusione
    cm = confusion_matrix(y_test, y_pred)
    print(f'Matrice di confusione:\n{cm}')
    return model

In [11]:
def test(df, model):
    
    df = ohe(df,['employment_status','payment_type','housing_status', 'source', 'device_os'])
    y_pred = model.predict(df.drop(['fraud_bool'], axis=1))
    y_test = df['fraud_bool']

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuratezza: {accuracy}')

    # Calcolare la precisione
    precision = precision_score(y_test, y_pred)
    print(f'Precisione: {precision}')

    # Calcolare il recall (sensibilità)
    recall = recall_score(y_test, y_pred)
    print(f'Recall: {recall}')

    # Calcolare FPR (False Positive Ration)
    print(f'FPR: {1 - recall}')

    # Matrice di confusione
    cm = confusion_matrix(y_test, y_pred)
    print(f'Matrice di confusione:\n{cm}')

    print('\n__________\n')

In [29]:

#Recall = TN / (TN + FP)
#FPR = FP / = 1 - Recall

0.849744711889132

# (75-25 con rapporto 1 a 50)

In [12]:
df = pd.read_csv("Base.csv").drop(['device_fraud_count', 'month'], axis=1)

In [13]:
df_frodi = df[(df['fraud_bool'] == 1)]
n = df[(df['fraud_bool'] == 1)].shape[0]
print(n)
df_not_frodi = df[(df['fraud_bool'] != 1)].head(n*50)

df = pd.concat([df_frodi, df_not_frodi])

from sklearn.model_selection import train_test_split
df = ohe(df,['employment_status','payment_type','housing_status', 'source', 'device_os'])
X_train, X_test, y_train, y_test = train_test_split(df.drop(['fraud_bool'], axis=1), df['fraud_bool'], random_state=0)

11029


## Train RandomForestClassifier

In [13]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

model = train(df, model)

Accuratezza: 0.9828402787654672
Precisione: 0.8063314711359404
Recall: 0.15791393143690738
FPR: 0.8420860685630926
Matrice di confusione:
[[137774    104]
 [  2309    433]]


## Train LogisticRegression

In [14]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model = train(df, model)

Accuratezza: 0.98061442184611
Precisione: 0.7105263157894737
Recall: 0.009846827133479213
FPR: 0.9901531728665208
Matrice di confusione:
[[137867     11]
 [  2715     27]]


c:\Users\Andrea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Train KNeighborsClassifier

In [15]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)

model = train(df, model)

Accuratezza: 0.9791281467785521
Precisione: 0.3748378728923476
Recall: 0.10539752005835157
FPR: 0.8946024799416484
Matrice di confusione:
[[137396    482]
 [  2453    289]]


## Train DecisionTreeClassifier

In [17]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

model = train(df, model)

Accuratezza: 0.9677997439908974
Precisione: 0.2209375
Recall: 0.2578409919766594
FPR: 0.7421590080233407
Matrice di confusione:
[[135385   2493]
 [  2035    707]]


## Train SVC

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')

model = train(df, model)

## Train GaussianNB

In [14]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

model = train(df, model)

Accuratezza: 0.8922628360119471
Precisione: 0.11360239162929746
Recall: 0.6652078774617067
FPR: 0.33479212253829327
Matrice di confusione:
[[123646  14232]
 [   918   1824]]


## Train LinearDiscriminantAnalysis

In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()

model = train(df, model)

Accuratezza: 0.9797752808988764
Precisione: 0.466796875
Recall: 0.2614879649890591
FPR: 0.7385120350109409
Matrice di confusione:
[[137059    819]
 [  2025    717]]


## Train AdaBoostClassifier

In [16]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()

model = train(df, model)

c:\Users\Andrea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuratezza: 0.9829327264969421
Precisione: 0.6734279918864098
Recall: 0.24215900802334062
FPR: 0.7578409919766593
Matrice di confusione:
[[137556    322]
 [  2078    664]]
